# SQL Context

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row

##### Create spark and sql context

In [3]:
spark = SparkContext("local", "sqlContext")
sql = SQLContext(spark)

##### Load data

In [9]:
# hardcoded
data = [('Alice', 1), ('Martha', 2), ('Anna', 3)]
data

[('Alice', 1), ('Martha', 2), ('Anna', 3)]

In [10]:
# from csv
data = spark.textFile('D:/data/csv/hello_world.csv')
data = data.filter(lambda x: x.split(',')).map(lambda line: line.split(','))

In [11]:
df = sql.createDataFrame(data, ['name', 'age'])
res = df.collect()
res

[Row(name='Alice', age='43'),
 Row(name='Martha', age='44'),
 Row(name='Anna', age='45')]

##### Parallelize

In [13]:
rdd = spark.parallelize(res)
df1 = sql.createDataFrame(rdd)
df1.collect()

[Row(name='Alice', age='43'),
 Row(name='Martha', age='44'),
 Row(name='Anna', age='45')]

In [15]:
df = sql.createDataFrame(rdd, ['name : string, age : int'])
res = df.collect()
res

[Row(name : string, age : int='Alice', age='43'),
 Row(name : string, age : int='Martha', age='44'),
 Row(name : string, age : int='Anna', age='45')]

##### Row

In [16]:
Person = Row('name', 'age')
person = rdd.map(lambda r: Person(*r))
df2 = sql.createDataFrame(person)
df2.collect()

[Row(name='Alice', age='43'),
 Row(name='Martha', age='44'),
 Row(name='Anna', age='45')]

##### Pandas

In [17]:
dfp = df1.toPandas()
dfp

,name,age
0,Alice,43
1,Martha,44
2,Anna,45


In [18]:
sql.createDataFrame(dfp).collect()  

[Row(name='Alice', age='43'),
 Row(name='Martha', age='44'),
 Row(name='Anna', age='45')]

##### SQL query

In [22]:
sql.registerDataFrameAsTable(df1, "table1")
df2 = sql.sql("SELECT table1.name AS Name, table1.age as Age FROM table1 WHERE table1.name='Martha'")
df2.collect()

[Row(Name='Martha', Age='44')]

In [24]:
sql.registerDataFrameAsTable(df1, "table1")
df2 = sql.table("table1")
#df2.collect()
df2.toPandas()

,name,age
0,Alice,43
1,Martha,44
2,Anna,45


## SQL Transformer

In [4]:
from pyspark.ml.feature import SQLTransformer

df = sql.createDataFrame([(0, 1.0, 3.0),(2, 2.0, 5.0)], ["id", "v1", "v2"])
df.show()

sqlTrans = SQLTransformer(statement="SELECT *, (v1 + v2) AS v3, (v1 * v2) AS v4 FROM __THIS__")
sqlTrans.transform(df).show()

+---+---+---+
| id| v1| v2|
+---+---+---+
|  0|1.0|3.0|
|  2|2.0|5.0|
+---+---+---+

+---+---+---+---+----+
| id| v1| v2| v3|  v4|
+---+---+---+---+----+
|  0|1.0|3.0|4.0| 3.0|
|  2|2.0|5.0|7.0|10.0|
+---+---+---+---+----+



In [32]:
spark.stop()

##### Credits & Links

https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.SQLContext